# TensorFlow变量

TensorFlow变量是表示被你的程序操作的、共享和可持久化的状态的最好方法。

变量通过`tf.Variable`类来操作。`tf.Variable`表示它的值可以通过它上面运行操作改变的张量。具体操作包括读取和修改张量的值。高层`tf.keras`使用`tf.Variable`来保存模型参数。这部分介绍如何在TensorFlow中创建、更新和管理`tf.Variable`。

## 创建变量

创建变量并提供初始值：

In [ ]:
import tensorflow as tf

my_variable = tf.Variable(tf.zeros([1, 2, 3]))

这创建一个全0填充的形状为\[1, 2, 3]的3维张量。没有指定类型，这个张量缺省类型dtype为`tf.float32`。若dtype没有指定，类型从张量初始值中推断。

如果指定`tf.device`，变量将放在那个设备上，否则，变量将放到最快的和它类型兼容的设备上（这意味着大多数变量自动的放在GPU上，如果有GPU的话）。例如，下列代码创建一个名字为v的变量，并将它放到第二个GPU设备上：

In [ ]:
with tf.device("/device:GPU:1"):
  v = tf.Variable(tf.zeros([10, 10]))

理想情况下，你应该使用`tf.distribute`API，那将允许你写一次代码，并让代码在不同的分布式设置中运行。

## 使用变量

要在TensorFlow图中使用`tf.Variable`的值，将它看作是一个一般的`tf.Tensor`:

In [ ]:
v = tf.Variable(0.0)
w = v + 1  # w是一个基于v的值计算得到的tf.Tensor
           # 任何时候一个变量在一个表达式中使用，它将自动的转换成表示它的值的tf.Tensor
print(w)

为了访问一个变量的值，使用方法`tf.Variable`类中的方法`assign`, `assign_add`等. 下面是这些方法的调用:

In [ ]:
v = tf.Variable(0.0)
v.assign_add(1)

多数TensorFlow优化器有优化的操作根据梯度下降一类的算法高效的更新变量的值。

你可以显式的读取变量的当前值，使用`read_value`:

In [ ]:
v = tf.Variable(0.0)
v.assign_add(1)
v.read_value()  # 1.0

当`tf.Variable`的最后一个引用离开作用域范围，它的内存释放。

## 跟踪变量

TensorFlow变量是一个Python对象。当你构建你自己的层、模型、优化器和其他相关工具时，你可能需要获得一个模型中所有变量的列表。

一个常见用例时实现`Layer`子类，Layer类递归的跟踪所有变量，并作为它的实例属性：

In [ ]:
class MyLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(MyLayer, self).__init__()
    self.my_var = tf.Variable(1.0)
    self.my_var_list = [tf.Variable(x) for x in range(10)]

class MyOtherLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(MyOtherLayer, self).__init__()
    self.sublayer = MyLayer()
    self.my_other_var = tf.Variable(10.0)

m = MyOtherLayer()
print(len(m.variables))  # 12 (11个来自 MyLayer, 1个来自MyOtherLayer)

如果你不是在开发一个新的层，TensorFlow也有一个一般的`tf.Module`基类，它只实现变量跟踪。`tf.Module`的实例有`variables`和`trainable_variables`属性，它们表示来自于模型的变量和可训练的变量。像Layer类一样，它通过遍历其他模块得到。